pip install --force-reinstall openai==1.8

In [1]:
from datasets import load_dataset
import pandas as pd
from rouge import Rouge
from openai import OpenAI
import json
from settings import *
from utils import *
import os 

from langchain.llms import OpenAI
from langchain import HuggingFaceHub, LLMChain
from langchain.prompts import load_prompt, PromptTemplate
from tqdm import tqdm
config = load_json(CONFIG_DIR)

In [2]:
# pubmed = load_data(config['data_path'])
# pubmed.head(5)
# filter_pubmed = pubmed[pubmed['article'].map(lambda x: len(x)) < 4000]
# filter_pubmed.to_csv('filter_pubmed.csv', index = False)

,article,abstract
0,a recent systematic analysis showed that in 20...,background : the present study was carried out...
1,it occurs in more than 50% of patients and may...,backgroundanemia in patients with cancer who a...
2,"tardive dystonia ( td ) , a rarer side effect ...",tardive dystonia ( td ) is a serious side effe...
3,"lepidoptera include agricultural pests that , ...",many lepidopteran insects are agricultural pes...
4,syncope is caused by transient diffuse cerebra...,we present an unusual case of recurrent cough ...


In [2]:
filter_pubmed = pd.read_csv('filter_pubmed.csv')
filter_pubmed.head(3)
filter_pubmed = filter_pubmed.dropna()
filter_pubmed.head(3)

,article,abstract
1,a review of the literature and an extensive me...,backgrounda review of the literature and an ex...
4,"nathan , as an oncology fellow , knew well tha...",t cells tell macrophages when to start making ...
5,temporary henna tattoos or pseudotattoo have b...,temporary henna tattoos or pseudotattoos have ...


In [3]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = load_json(API_KEY)['huggingface_key']
summarizer = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.2}
)
template = """
You are an abstractive summarizer that follows the output pattern.
Please revise the extracted summary based on the document. The revised summary should include the information in the extracted summary. Docu- ment: {document} Extractive Summary: [Extractive Summary] [Format Instruction].
"""

response_list = []
for idx in tqdm(range(10)):
    doc = filter_pubmed.iloc[idx, 0]
    prompt = PromptTemplate(template=template, input_variables=["document"])
    llm_chain = LLMChain(prompt=prompt, llm=summarizer)
    response = llm_chain.invoke(input = doc)['document']
    if len(response) > 0:
        response_list.append([response, filter_pubmed.iloc[idx, 1]])
    

/opt/conda/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)
100%|██████████| 10/10 [01:22<00:00,  8.24s/it]


In [4]:
for i in response_list:
    print(get_Rouge_score(*i))

{'rouge-1': {'r': 0.972972972972973, 'p': 0.4268774703557312, 'f': 0.5934065891675221}, 'rouge-2': {'r': 0.9259259259259259, 'p': 0.3424657534246575, 'f': 0.49999999605800005}, 'rouge-l': {'r': 0.972972972972973, 'p': 0.4268774703557312, 'f': 0.5934065891675221}}
{'rouge-1': {'r': 0.4864864864864865, 'p': 0.07003891050583658, 'f': 0.12244897739159614}, 'rouge-2': {'r': 0.1590909090909091, 'p': 0.015217391304347827, 'f': 0.027777776184177465}, 'rouge-l': {'r': 0.4594594594594595, 'p': 0.06614785992217899, 'f': 0.11564625630316074}}
{'rouge-1': {'r': 0.8035714285714286, 'p': 0.3157894736842105, 'f': 0.4534004997278075}, 'rouge-2': {'r': 0.5654761904761905, 'p': 0.17857142857142858, 'f': 0.27142856778057145}, 'rouge-l': {'r': 0.8035714285714286, 'p': 0.3157894736842105, 'f': 0.4534004997278075}}
{'rouge-1': {'r': 0.9081632653061225, 'p': 1.0, 'f': 0.9518716527655925}, 'rouge-2': {'r': 0.8988095238095238, 'p': 0.967948717948718, 'f': 0.9320987604389576}, 'rouge-l': {'r': 0.9081632653061225